Importar librerias


In [ ]:
import pandas as pd #libreria de python para manipular datos
import numpy as np # libreria de python para calculos numericos y analisis
import os #modulo para usar funcionalidades del sistema operativo
from pathlib import Path # modulo de la ruta que se usa
from heapq import merge # modulo para unir Dataframes
import oracledb
from ipynb.fs.defs.ConexionBD import ConectarBD
import re
import datetime

pd.set_option('display.max_columns',None)#Cuantas columnas mostar por Dataframe

In [ ]:
separador = os.path.sep #busca el separador que sería / o \
current_dir = os.path.dirname(os.path.realpath('CruceUsuariosBD.ipynb'))  #busca la ruta del archivo en el que está CruceUsuariosAPP.ipynb
dir = separador.join(current_dir.split(separador)[:-1]) #se devuelve una carpeta
filenameT = os.path.join(dir, "Data\\BD")
filenameA = os.path.join(dir, "Data\\APP") #se ruta para leer los export de los aplicativos
filename = os.path.join(dir, "Resultado\\Primario") 
filenameS= os.path.join(dir,"Data\\USUARIOS")

Importar usuarios

In [ ]:
file_exists_Usuarios = os.path.exists(filename+"\\IdentidadesEncontradasconSSFF.csv")
print(file_exists_Usuarios) #muestra false o true 

if file_exists_Usuarios == True: #Si el archivo existe

    Usuarios=pd.read_csv(filename+"\\IdentidadesEncontradasconSSFF.csv", sep=';', encoding = "utf-8-sig" , engine='python') #importa el archivo csv separado por punto y comas
    
    Usuarios= Usuarios[['DocumentoRRHH', 'Identidad','NombreRRHH', 'CASO', 'TIPO DE CONTRATO','FECHA DE RETIRO', 'Usuario_Aplicativo', 'Aplica Inactivacion']]
    Usuarios = Usuarios[Usuarios['Aplica Inactivacion']=="si"]
    
    Usuarios = Usuarios.drop('Usuario_Aplicativo',  axis=1)
    Usuarios = Usuarios.drop('Aplica Inactivacion',  axis=1)
    Usuarios["FECHA DE RETIRO"]=Usuarios["FECHA DE RETIRO"].astype(str).str.replace(" 00:00:00", "", regex=True)
    #Usuarios['DocumentoRRHH'] = Usuarios['DocumentoRRHH'].astype('Int64')
    Usuarios["DocumentoRRHH"] = pd.to_numeric(Usuarios["DocumentoRRHH"], errors='ignore')
    Usuarios['DocumentoRRHH'] = Usuarios['DocumentoRRHH'].astype(str)
    Usuarios["FECHA DE RETIRO"]=Usuarios["FECHA DE RETIRO"].astype(str).str.replace(" 0:00", "", regex=True)
    # Usuarios['DocumentoRRHH'] = [ele.rstrip(".0") for ele in Usuarios['DocumentoRRHH']]
    Usuarios=Usuarios.drop_duplicates() #Quita filas que tengan el mismo documento
    Identidades=Usuarios[['Identidad']]
    Identidades=Identidades.drop_duplicates()
    Identidades=Identidades['Identidad'].tolist()
    if "sin identidad" in Identidades:
        Identidades.remove("sin identidad")
    Nombres1=Usuarios[['NombreRRHH']]
    Nombres1=Nombres1['NombreRRHH'].tolist()
    name = Usuarios["NombreRRHH"].str.split( n=3, expand=True)
    name= pd.DataFrame(name)
    name.columns = ['nombre1', 'nombre2','apellido1', 'apellido2']

    
    name = name.fillna({'nombre1': '1', 'nombre2': '2','apellido1': '3', 'apellido2': '4'})
    
    name['LLAVE1'] = name["apellido1"] + " " + name["apellido2"]+ " " + name["nombre1"]+ " " + name["nombre2"]
    name['LLAVE2'] = name["nombre2"] + " " + name["apellido1"]+ " " + name["nombre1"]
    name = name.replace("  ", " ", regex=True)
    llavesnom = pd.concat([Usuarios, name], axis=1)
    llavesnom= llavesnom.drop(['Identidad', 'nombre1', 'nombre2', 'apellido1', 'apellido2'], axis=1)
    Nombres2=name['LLAVE1'].tolist()
    Nombres3=name['LLAVE2'].tolist()

In [ ]:
Usuarios

Importar LNXRACDBPR01

In [ ]:
file_exists_CA = os.path.exists(filenameT+"\\CA - LNXRACDBPR01.xlsx")

print(file_exists_CA)

if file_exists_CA == True:
    ca=pd.read_excel(filenameT+"\\CA - LNXRACDBPR01.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    ca.drop(ca.index[0:11], inplace=True)
    ca= ca.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    ca['Estado'] = ca['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    ca['Estado'] = ca['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    ca['Estado'] = ca['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    ca.loc[ca['Estado'] != 'Activo','Estado']='Inactivo'
    ca["Identidad"] = ca["Identidad"].str.lower()
    ca['Identidad'] = ca['Identidad'].astype(str)
    ca['Identidad'] = [ele.lstrip(' ') for ele in ca['Identidad']]
    ca['Identidad'] = [ele.rstrip(' ') for ele in ca['Identidad']]
    ca['Identidad'] = [ele.lstrip('.') for ele in ca['Identidad']]
    ca['Identidad'] = [ele.rstrip('.') for ele in ca['Identidad']]
    filtroId = ca[ca.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    ca = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    ca.index = range(ca.shape[0])
    ca=ca.assign(Sistema='CA') # se agrega una columna para saber que ese filtro fue de ClaroCo
    ca=ca.assign(Nombre_BD='LNXRACDBPR01')
    ca =ca.assign(Nombre='Sin nombre')
    ca= ca[[ 'Identidad','Estado','Cruce','Nombre', 'Sistema','Nombre_BD','DocumentoRRHH','NombreRRHH', 'CASO', 'FECHA DE RETIRO','TIPO DE CONTRATO']]

Importar OIM - OIM1_comsun68

In [ ]:
file_exists_OIM = os.path.exists(filenameT+"\\OIM - OIM1_comsun68.xlsx")

print(file_exists_OIM)

if file_exists_OIM == True:
    oim=pd.read_excel(filenameT+"\\OIM - OIM1_comsun68.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    oim.drop(oim.index[0:11], inplace=True)
    oim= oim.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    oim['Estado'] = oim['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    oim['Estado'] = oim['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    oim['Estado'] = oim['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    oim.loc[oim['Estado'] != 'Activo','Estado']='Inactivo'
    oim["Identidad"] = oim["Identidad"].str.lower()
    oim['Identidad'] = oim['Identidad'].astype(str).str.replace(".", "", regex=True)
    oim['Identidad'] = [ele.lstrip(' ') for ele in oim['Identidad']]
    oim['Identidad'] = [ele.rstrip(' ') for ele in oim['Identidad']]
    oim['Identidad'] = [ele.lstrip('.') for ele in oim['Identidad']]
    oim['Identidad'] = [ele.rstrip('.') for ele in oim['Identidad']]
    filtroId = oim[oim.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    oim = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    oim.index = range(oim.shape[0])
    oim=oim.assign(Sistema='OIM') # se agrega una columna para saber que ese filtro fue de ClaroCo
    oim=oim.assign(Nombre_BD='OIM1')
    oim =oim.assign(Nombre='Sin nombre')
    Nube = pd.concat([ca, oim])
    Nube.index = range(Nube.shape[0])

Importar BSCSPROD

In [ ]:
midb=ConectarBD()

In [ ]:

if not midb.empty:

    bscsprod=midb[['USERNAME','DESCRIPTION','BLOQUEADO ORACLE']] #USAR SOLO ESAS TRES COLUMNAS
    nom = bscsprod["DESCRIPTION"].str.split("-", n=2, expand=True)
    nom= pd.DataFrame(nom)
    nom.columns = ['1', '2', '3']
    bscsprod = pd.concat([bscsprod, nom], axis=1)
    bscsprod.rename(columns={'USERNAME':'Identidad', '1':'Nombre', 'BLOQUEADO ORACLE':'Estado'}, inplace=True)
    bscsprod=bscsprod[['Identidad','Nombre','Estado']]
    bscsprod["Nombre"]= bscsprod["Nombre"].str.lower()
    bscsprod["Identidad"]= bscsprod["Identidad"].str.lower()
    bscsprod['Identidad'] = [ele.lstrip(' ') for ele in bscsprod['Identidad']]
    bscsprod['Identidad'] = [ele.rstrip(' ') for ele in bscsprod['Identidad']]
    bscsprod['Identidad'] = [ele.lstrip('.') for ele in bscsprod['Identidad']]
    bscsprod['Identidad'] = [ele.rstrip('.') for ele in bscsprod['Identidad']]
    bscsprod['Estado'] = bscsprod['Estado'].astype(str).str.replace("NO", "Activo", regex=True)
    bscsprod.loc[bscsprod['Estado'] != 'Activo','Estado']='Inactivo'
    
    bscsprod['Nombre'] = bscsprod['Nombre'].astype(str).str.replace(".", "", regex=True)
    bscsprod['Nombre'] = bscsprod['Nombre'].astype(str).str.replace("á", "a", regex=True)
    bscsprod['Nombre'] = bscsprod['Nombre'].astype(str).str.replace("é", "e", regex=True)
    bscsprod['Nombre'] = bscsprod['Nombre'].astype(str).str.replace("í", "i", regex=True)
    bscsprod['Nombre'] = bscsprod['Nombre'].astype(str).str.replace("ó", "o", regex=True)
    bscsprod['Nombre'] = bscsprod['Nombre'].astype(str).str.replace("ú", "u", regex=True)
    bscsprod = bscsprod.fillna({ 'Nombre': 'Sin nombre'})

    filtroId = bscsprod[bscsprod.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    filtroId = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')

    filtro2 = bscsprod[bscsprod.Nombre.isin(Nombres1)]#filtra por nombre completo
    filtro2=filtro2.assign(Cruce='Nombre completo')
    filtro2 = filtro2.merge(llavesnom, how='left', left_on='Nombre', right_on='NombreRRHH')
    #filtro2.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro3 = bscsprod[bscsprod.Nombre.isin(Nombres2)]#filtra llave2
    filtro3=filtro3.assign(Cruce='Llave 1')
    filtro3 = filtro3.merge(llavesnom, how='left', left_on='Nombre', right_on='LLAVE1')
    #filtro3.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    filtro4 = bscsprod[bscsprod.Nombre.isin(Nombres3)]#filtra llave2
    filtro4=filtro4.assign(Cruce='Llave 2')
    filtro4 = filtro4.merge(llavesnom, how='left', left_on='Nombre', right_on='LLAVE2')
    #filtro4.rename(columns={'Documento1':'DocumentoRRHH'}, inplace=True)
    validaNombres= pd.concat([filtro2, filtro3])
    validaNombres= pd.concat([validaNombres, filtro4])
    validaNombres=validaNombres.assign(Espacios=validaNombres['Nombre'].str.count(' '))
    validaNombres = validaNombres.drop(validaNombres[validaNombres['Espacios']==1].index)
    validaNombres = validaNombres.drop('Espacios', axis=1)
    validaNombres= validaNombres.drop(['LLAVE1', 'LLAVE2'], axis=1)
    bscsprod=pd.concat([filtroId, validaNombres])
    
    bscsprod= bscsprod.drop_duplicates(bscsprod.columns[~bscsprod.columns.isin(['Cruce'])], keep='first')
    bscsprod.index = range(bscsprod.shape[0])
    bscsprod=bscsprod.assign(Sistema='AC - BSCS - MOTOR DE PAGOS') # se agrega una columna para saber que ese filtro fue de ClaroCo
    bscsprod=bscsprod.assign(Nombre_BD='BSCSPROD')
    Nube = pd.concat([Nube, bscsprod])
    Nube.index = range(Nube.shape[0])
    Nube= Nube[[ 'Identidad','Estado','Cruce','Nombre', 'Sistema','Nombre_BD','DocumentoRRHH','NombreRRHH', 'CASO', 'FECHA DE RETIRO','TIPO DE CONTRATO']]

CALLIDUS - CALDMPROD

In [ ]:
file_exists_caldmprod = os.path.exists(filenameT+"\\CALLIDUS - CALDMPROD.xlsx")

print(file_exists_caldmprod)

if file_exists_caldmprod == True:
    caldmprod=pd.read_excel(filenameT+"\\CALLIDUS - CALDMPROD.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    caldmprod.drop(caldmprod.index[0:11], inplace=True)
    caldmprod= caldmprod.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    caldmprod['Estado'] = caldmprod['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    caldmprod['Estado'] = caldmprod['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    caldmprod['Estado'] = caldmprod['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    caldmprod.loc[caldmprod['Estado'] != 'Activo','Estado']='Inactivo'
    caldmprod['Identidad'] = caldmprod['Identidad'].astype(str)
    caldmprod["Identidad"] = caldmprod["Identidad"].str.lower()
    caldmprod['Identidad'] = [ele.lstrip('.') for ele in caldmprod['Identidad']]
    caldmprod['Identidad'] = [ele.rstrip('.') for ele in caldmprod['Identidad']]
    caldmprod['Identidad'] = [ele.lstrip(' ') for ele in caldmprod['Identidad']]
    caldmprod['Identidad'] = [ele.rstrip(' ') for ele in caldmprod['Identidad']]
    filtroId = caldmprod[caldmprod.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    caldmprod = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    caldmprod.index = range(caldmprod.shape[0])
    caldmprod=caldmprod.assign(Sistema='CALLIDUS') # se agrega una columna para saber que ese filtro fue de ClaroCo
    caldmprod=caldmprod.assign(Nombre_BD='CALDMPROD')
    caldmprod =caldmprod.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, caldmprod])
    Nube.index = range(Nube.shape[0])

CALLIDUS - TRUECPROD

In [ ]:
file_exists_truecprod = os.path.exists(filenameT+"\\CALLIDUS - TRUECPROD.xlsx")

print(file_exists_truecprod)

if file_exists_truecprod == True:
    truecprod=pd.read_excel(filenameT+"\\CALLIDUS - TRUECPROD.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    truecprod.drop(truecprod.index[0:11], inplace=True)
    truecprod= truecprod.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    truecprod['Estado'] = truecprod['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    truecprod['Estado'] = truecprod['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    truecprod['Estado'] = truecprod['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    truecprod.loc[truecprod['Estado'] != 'Activo','Estado']='Inactivo'
    truecprod['Identidad'] = truecprod['Identidad'].astype(str)
    truecprod["Identidad"] = truecprod["Identidad"].str.lower()
    truecprod['Identidad'] = [ele.lstrip('.') for ele in truecprod['Identidad']]
    truecprod['Identidad'] = [ele.rstrip('.') for ele in truecprod['Identidad']]
    truecprod['Identidad'] = [ele.lstrip(' ') for ele in truecprod['Identidad']]
    truecprod['Identidad'] = [ele.rstrip(' ') for ele in truecprod['Identidad']]
    filtroId = truecprod[truecprod.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    truecprod = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    truecprod.index = range(truecprod.shape[0])
    truecprod=truecprod.assign(Sistema='CALLIDUS') # se agrega una columna para saber que ese filtro fue de ClaroCo
    truecprod=truecprod.assign(Nombre_BD='TRUECPROD')
    truecprod =truecprod.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, truecprod])
    Nube.index = range(Nube.shape[0])

CALLIDUS - LOMBAPROD

In [ ]:
file_exists_lombaprod = os.path.exists(filenameT+"\\CALLIDUS - LOMBAPROD.xlsx")

print(file_exists_lombaprod)

if file_exists_lombaprod == True:
    lombaprod=pd.read_excel(filenameT+"\\CALLIDUS - LOMBAPROD.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    lombaprod.drop(lombaprod.index[0:11], inplace=True)
    lombaprod= lombaprod.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    lombaprod['Estado'] = lombaprod['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    lombaprod['Estado'] = lombaprod['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    lombaprod['Estado'] = lombaprod['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    lombaprod.loc[lombaprod['Estado'] != 'Activo','Estado']='Inactivo'
    lombaprod['Identidad'] = lombaprod['Identidad'].astype(str)
    lombaprod["Identidad"] = lombaprod["Identidad"].str.lower()
    lombaprod['Identidad'] = [ele.lstrip('.') for ele in lombaprod['Identidad']]
    lombaprod['Identidad'] = [ele.rstrip('.') for ele in lombaprod['Identidad']]
    lombaprod['Identidad'] = [ele.lstrip(' ') for ele in lombaprod['Identidad']]
    lombaprod['Identidad'] = [ele.rstrip(' ') for ele in lombaprod['Identidad']]
    filtroId = lombaprod[lombaprod.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    lombaprod = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    lombaprod.index = range(lombaprod.shape[0])
    lombaprod=lombaprod.assign(Sistema='CALLIDUS') # se agrega una columna para saber que ese filtro fue de ClaroCo
    lombaprod=lombaprod.assign(Nombre_BD='LOMBAPROD')
    lombaprod =lombaprod.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, lombaprod])
    Nube.index = range(Nube.shape[0])

CALLIDUS - BUSOBPROD.xlsx

In [ ]:
file_exists_busobprod = os.path.exists(filenameT+"\\CALLIDUS - BUSOBPROD.xlsx")

print(file_exists_busobprod)

if file_exists_busobprod == True:
    busobprod=pd.read_excel(filenameT+"\\CALLIDUS - BUSOBPROD.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    busobprod.drop(busobprod.index[0:11], inplace=True)
    busobprod= busobprod.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    busobprod['Estado'] = busobprod['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    busobprod['Estado'] = busobprod['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    busobprod['Estado'] = busobprod['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    busobprod.loc[busobprod['Estado'] != 'Activo','Estado']='Inactivo'
    busobprod['Identidad'] = busobprod['Identidad'].astype(str)
    busobprod["Identidad"] = busobprod["Identidad"].str.lower()
    busobprod['Identidad'] = [ele.lstrip(' ') for ele in busobprod['Identidad']]
    busobprod['Identidad'] = [ele.rstrip(' ') for ele in busobprod['Identidad']]
    busobprod['Identidad'] = [ele.lstrip('.') for ele in busobprod['Identidad']]
    busobprod['Identidad'] = [ele.rstrip('.') for ele in busobprod['Identidad']]
    filtroId = busobprod[busobprod.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 

    
    busobprod = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    busobprod.index = range(busobprod.shape[0])
    busobprod=busobprod.assign(Sistema='CALLIDUS') # se agrega una columna para saber que ese filtro fue de ClaroCo
    busobprod=busobprod.assign(Nombre_BD='BUSOBPROD')
    busobprod =busobprod.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, busobprod])
    Nube.index = range(Nube.shape[0])

CRM - ONIX

In [ ]:
file_exists_crmonix = os.path.exists(filenameT+"\\CRM - ONIX.xlsx")

print(file_exists_crmonix)

if file_exists_crmonix == True:
    crmonix=pd.read_excel(filenameT+"\\CRM - ONIX.xlsx", names=['1', '2', '3', '4','5','Identidad','7','8','9','10', '11', '12', '13', '14', 'Estado'])
    crmonix.drop(crmonix.index[0:2], inplace=True)
    crmonix= crmonix.drop(['1','2','3', '4', '5', '7', '8', '9', '9', '10', '11', '12', '13', '14'], axis=1)
    crmonix['Estado'] = crmonix['Estado'].astype(str).str.replace("Deshabilitado", "Inactivo", regex=True)
    crmonix['Estado'] = crmonix['Estado'].astype(str).str.replace("Habilitado", "Activo", regex=True)
    crmonix["Identidad"] = crmonix["Identidad"].str.lower()
    crmonix['Identidad'] = [ele.lstrip(' ') for ele in crmonix['Identidad']]
    crmonix['Identidad'] = [ele.rstrip(' ') for ele in crmonix['Identidad']]
    crmonix['Identidad'] = [ele.lstrip('.') for ele in crmonix['Identidad']]
    crmonix['Identidad'] = [ele.rstrip('.') for ele in crmonix['Identidad']]
    filtroId = crmonix[crmonix.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    crmonix = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    crmonix.index = range(crmonix.shape[0])
    crmonix=crmonix.assign(Sistema='CRM-ONYX') # se agrega una columna para saber que ese filtro fue de ClaroCo
    crmonix=crmonix.assign(Nombre_BD='ONYX')
    crmonix =crmonix.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, crmonix])
    Nube.index = range(Nube.shape[0])

INTEC - WBMSDBPR

In [ ]:
file_exists_wbmsdbpr = os.path.exists(filenameT+"\\INTEC - WBMSDBPR.xlsx")

print(file_exists_wbmsdbpr)

if file_exists_wbmsdbpr == True:
    wbmsdbpr=pd.read_excel(filenameT+"\\INTEC - WBMSDBPR.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    wbmsdbpr.drop(wbmsdbpr.index[0:11], inplace=True)
    wbmsdbpr= wbmsdbpr.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    wbmsdbpr['Estado'] = wbmsdbpr['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    wbmsdbpr['Estado'] = wbmsdbpr['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    wbmsdbpr['Estado'] = wbmsdbpr['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    wbmsdbpr.loc[wbmsdbpr['Estado'] != 'Activo','Estado']='Inactivo'
    wbmsdbpr['Identidad'] = wbmsdbpr['Identidad'].astype(str)
    wbmsdbpr["Identidad"] = wbmsdbpr["Identidad"].str.lower()
    wbmsdbpr['Identidad'] = [ele.lstrip('.') for ele in wbmsdbpr['Identidad']]
    wbmsdbpr['Identidad'] = [ele.rstrip('.') for ele in wbmsdbpr['Identidad']]
    wbmsdbpr['Identidad'] = [ele.lstrip(' ') for ele in wbmsdbpr['Identidad']]
    wbmsdbpr['Identidad'] = [ele.rstrip(' ') for ele in wbmsdbpr['Identidad']]
    filtroId = wbmsdbpr[wbmsdbpr.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    wbmsdbpr = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    wbmsdbpr.index = range(wbmsdbpr.shape[0])
    wbmsdbpr=wbmsdbpr.assign(Sistema='INTEC') # se agrega una columna para saber que ese filtro fue de ClaroCo
    wbmsdbpr=wbmsdbpr.assign(Nombre_BD='WBMSDBPR')
    wbmsdbpr =wbmsdbpr.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, wbmsdbpr])
    Nube.index = range(Nube.shape[0])

INTEGRADOR WEB - INTEGPRD 70

In [ ]:
file_exists_integprd70 = os.path.exists(filenameT+"\\INTEGRADOR WEB - INTEGPRD 70.xlsx")

print(file_exists_integprd70)

if file_exists_integprd70 == True:
    integprd70=pd.read_excel(filenameT+"\\INTEGRADOR WEB - INTEGPRD 70.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    integprd70.drop(integprd70.index[0:11], inplace=True)
    integprd70= integprd70.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    integprd70['Estado'] = integprd70['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    integprd70['Estado'] = integprd70['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    integprd70['Estado'] = integprd70['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    integprd70.loc[integprd70['Estado'] != 'Activo','Estado']='Inactivo'
    integprd70['Identidad'] = integprd70['Identidad'].astype(str)
    integprd70["Identidad"] = integprd70["Identidad"].str.lower()
    integprd70['Identidad'] = [ele.lstrip('.') for ele in integprd70['Identidad']]
    integprd70['Identidad'] = [ele.rstrip('.') for ele in integprd70['Identidad']]
    integprd70['Identidad'] = [ele.lstrip(' ') for ele in integprd70['Identidad']]
    integprd70['Identidad'] = [ele.rstrip(' ') for ele in integprd70['Identidad']]
    filtroId = integprd70[integprd70.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    integprd70 = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    integprd70.index = range(integprd70.shape[0])
    integprd70=integprd70.assign(Sistema='INTEGRADOR WEB') # se agrega una columna para saber que ese filtro fue de ClaroCo
    integprd70=integprd70.assign(Nombre_BD='INTEGPRD 70')
    integprd70 =integprd70.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, integprd70])
    Nube.index = range(Nube.shape[0])

INTEGRADOR WEB - INTEGPRD 71

In [ ]:
file_exists_integprd71 = os.path.exists(filenameT+"\\INTEGRADOR WEB - INTEGPRD 71.xlsx")

print(file_exists_integprd71)

if file_exists_integprd71 == True:
    integprd71=pd.read_excel(filenameT+"\\INTEGRADOR WEB - INTEGPRD 71.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    integprd71.drop(integprd71.index[0:11], inplace=True)
    integprd71= integprd71.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    integprd71['Estado'] = integprd71['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    integprd71['Estado'] = integprd71['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    integprd71['Estado'] = integprd71['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    integprd71.loc[integprd71['Estado'] != 'Activo','Estado']='Inactivo'
    integprd71['Identidad'] = integprd71['Identidad'].astype(str)
    integprd71["Identidad"] = integprd71["Identidad"].str.lower()
    integprd71['Identidad'] = [ele.lstrip('.') for ele in integprd71['Identidad']]
    integprd71['Identidad'] = [ele.rstrip('.') for ele in integprd71['Identidad']]
    integprd71['Identidad'] = [ele.lstrip(' ') for ele in integprd71['Identidad']]
    integprd71['Identidad'] = [ele.rstrip(' ') for ele in integprd71['Identidad']]
    filtroId = integprd71[integprd71.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    integprd71 = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    integprd71.index = range(integprd71.shape[0])
    integprd71=integprd71.assign(Sistema='INTEGRADOR WEB') # se agrega una columna para saber que ese filtro fue de ClaroCo
    integprd71=integprd71.assign(Nombre_BD='INTEGPRD 71')
    integprd71 =integprd71.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, integprd71])
    Nube.index = range(Nube.shape[0])

MAGUARE  - GPRS.xlsx

In [ ]:
file_exists_gprs = os.path.exists(filenameT+"\\MAGUARE  - GPRS.xlsx")

print(file_exists_gprs)

if file_exists_gprs == True:
    gprs=pd.read_excel(filenameT+"\\MAGUARE  - GPRS.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    gprs.drop(gprs.index[0:11], inplace=True)
    gprs= gprs.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    gprs['Estado'] = gprs['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    gprs['Estado'] = gprs['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    gprs['Estado'] = gprs['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    gprs.loc[gprs['Estado'] != 'Activo','Estado']='Inactivo'
    gprs['Identidad'] = gprs['Identidad'].astype(str)
    gprs["Identidad"] = gprs["Identidad"].str.lower()
    gprs['Identidad'] = [ele.lstrip('.') for ele in gprs['Identidad']]
    gprs['Identidad'] = [ele.rstrip('.') for ele in gprs['Identidad']]
    gprs['Identidad'] = [ele.lstrip(' ') for ele in gprs['Identidad']]
    gprs['Identidad'] = [ele.rstrip(' ') for ele in gprs['Identidad']]
    filtroId = gprs[gprs.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    gprs = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    gprs.index = range(gprs.shape[0])
    gprs=gprs.assign(Sistema='MAGUARE') # se agrega una columna para saber que ese filtro fue de ClaroCo
    gprs=gprs.assign(Nombre_BD='GPRS')
    gprs =gprs.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, gprs])
    Nube.index = range(Nube.shape[0])

MAHINDRA - PRTPCOL1_comibm108

In [ ]:
file_exists_comibm108 = os.path.exists(filenameT+"\\MAHINDRA - PRTPCOL1_comibm108.xlsx")

print(file_exists_comibm108)

if file_exists_comibm108 == True:
    comibm108=pd.read_excel(filenameT+"\\MAHINDRA - PRTPCOL1_comibm108.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    comibm108.drop(comibm108.index[0:11], inplace=True)
    comibm108= comibm108.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    comibm108['Estado'] = comibm108['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    comibm108['Estado'] = comibm108['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    comibm108['Estado'] = comibm108['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    comibm108.loc[comibm108['Estado'] != 'Activo','Estado']='Inactivo'
    comibm108['Identidad'] = comibm108['Identidad'].astype(str)
    comibm108["Identidad"] = comibm108["Identidad"].str.lower()
    comibm108['Identidad'] = [ele.lstrip('.') for ele in comibm108['Identidad']]
    comibm108['Identidad'] = [ele.rstrip('.') for ele in comibm108['Identidad']]
    comibm108['Identidad'] = [ele.lstrip(' ') for ele in comibm108['Identidad']]
    comibm108['Identidad'] = [ele.rstrip(' ') for ele in comibm108['Identidad']]
    filtroId = comibm108[comibm108.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    comibm108 = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    comibm108.index = range(comibm108.shape[0])
    comibm108=comibm108.assign(Sistema='MAHINDRA') # se agrega una columna para saber que ese filtro fue de ClaroCo
    comibm108=comibm108.assign(Nombre_BD='PRTPCOL1_COMIBM108')
    comibm108 =comibm108.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, comibm108])
    Nube.index = range(Nube.shape[0])

MAHINDRA - PRTPCOL2_comibm107

In [ ]:
file_exists_comibm107 = os.path.exists(filenameT+"\\MAHINDRA - PRTPCOL2_comibm107.xlsx")

print(file_exists_comibm107)

if file_exists_comibm107 == True:
    comibm107=pd.read_excel(filenameT+"\\MAHINDRA - PRTPCOL2_comibm107.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    comibm107.drop(comibm107.index[0:11], inplace=True)
    comibm107= comibm107.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    comibm107['Estado'] = comibm107['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    comibm107['Estado'] = comibm107['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    comibm107['Estado'] = comibm107['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    comibm107.loc[comibm107['Estado'] != 'Activo','Estado']='Inactivo'
    comibm107['Identidad'] = comibm107['Identidad'].astype(str)
    comibm107["Identidad"] = comibm107["Identidad"].str.lower()
    comibm107['Identidad'] = [ele.lstrip('.') for ele in comibm107['Identidad']]
    comibm107['Identidad'] = [ele.rstrip('.') for ele in comibm107['Identidad']]
    comibm107['Identidad'] = [ele.lstrip(' ') for ele in comibm107['Identidad']]
    comibm107['Identidad'] = [ele.rstrip(' ') for ele in comibm107['Identidad']]
    filtroId = comibm107[comibm107.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    comibm107 = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    comibm107.index = range(comibm107.shape[0])
    comibm107=comibm107.assign(Sistema='MAHINDRA') # se agrega una columna para saber que ese filtro fue de ClaroCo
    comibm107=comibm107.assign(Nombre_BD='PRTPCOL2_COMIBM107')
    comibm107 =comibm107.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, comibm107])
    Nube.index = range(Nube.shape[0])

MAHINDRA - ussdcol1

In [ ]:
file_exists_ussdcol1 = os.path.exists(filenameT+"\\MAHINDRA - ussdcol1.xlsx")

print(file_exists_ussdcol1)

if file_exists_ussdcol1 == True:
    ussdcol1=pd.read_excel(filenameT+"\\MAHINDRA - ussdcol1.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    ussdcol1.drop(ussdcol1.index[0:11], inplace=True)
    ussdcol1= ussdcol1.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    ussdcol1['Estado'] = ussdcol1['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    ussdcol1['Estado'] = ussdcol1['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    ussdcol1['Estado'] = ussdcol1['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    ussdcol1.loc[ussdcol1['Estado'] != 'Activo','Estado']='Inactivo'
    ussdcol1['Identidad'] = ussdcol1['Identidad'].astype(str)
    ussdcol1["Identidad"] = ussdcol1["Identidad"].str.lower()
    ussdcol1['Identidad'] = [ele.lstrip('.') for ele in ussdcol1['Identidad']]
    ussdcol1['Identidad'] = [ele.rstrip('.') for ele in ussdcol1['Identidad']]
    ussdcol1['Identidad'] = [ele.lstrip(' ') for ele in ussdcol1['Identidad']]
    ussdcol1['Identidad'] = [ele.rstrip(' ') for ele in ussdcol1['Identidad']]
    filtroId = ussdcol1[ussdcol1.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    ussdcol1 = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    ussdcol1.index = range(ussdcol1.shape[0])
    ussdcol1=ussdcol1.assign(Sistema='MAHINDRA') # se agrega una columna para saber que ese filtro fue de ClaroCo
    ussdcol1=ussdcol1.assign(Nombre_BD='USSDCOL1')
    ussdcol1 =ussdcol1.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, ussdcol1])
    Nube.index = range(Nube.shape[0])

MAHINDRA - ussdcol2

In [ ]:
file_exists_ussdcol2 = os.path.exists(filenameT+"\\MAHINDRA - ussdcol2.xlsx")

print(file_exists_ussdcol2)

if file_exists_ussdcol2 == True:
    ussdcol2=pd.read_excel(filenameT+"\\MAHINDRA - ussdcol2.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    ussdcol2.drop(ussdcol2.index[0:11], inplace=True)
    ussdcol2= ussdcol2.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    ussdcol2['Estado'] = ussdcol2['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    ussdcol2['Estado'] = ussdcol2['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    ussdcol2['Estado'] = ussdcol2['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    ussdcol2.loc[ussdcol2['Estado'] != 'Activo','Estado']='Inactivo'
    ussdcol2['Identidad'] = ussdcol2['Identidad'].astype(str)
    ussdcol2["Identidad"] = ussdcol2["Identidad"].str.lower()
    ussdcol2['Identidad'] = [ele.lstrip('.') for ele in ussdcol2['Identidad']]
    ussdcol2['Identidad'] = [ele.rstrip('.') for ele in ussdcol2['Identidad']]
    ussdcol2['Identidad'] = [ele.lstrip(' ') for ele in ussdcol2['Identidad']]
    ussdcol2['Identidad'] = [ele.rstrip(' ') for ele in ussdcol2['Identidad']]
    filtroId = ussdcol2[ussdcol2.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    ussdcol2 = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    ussdcol2.index = range(ussdcol2.shape[0])
    ussdcol2=ussdcol2.assign(Sistema='MAHINDRA') # se agrega una columna para saber que ese filtro fue de ClaroCo
    ussdcol2=ussdcol2.assign(Nombre_BD='USSDCOL2')
    ussdcol2 =ussdcol2.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, ussdcol2])
    Nube.index = range(Nube.shape[0])

MOTOR DE PAGOS - DATAREP

In [ ]:
file_exists_datarep = os.path.exists(filenameT+"\\MOTOR DE PAGOS - DATAREP.xlsx")

print(file_exists_datarep)

if file_exists_datarep == True:
    datarep=pd.read_excel(filenameT+"\\MOTOR DE PAGOS - DATAREP.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    datarep.drop(datarep.index[0:11], inplace=True)
    datarep= datarep.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    datarep['Estado'] = datarep['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    datarep['Estado'] = datarep['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    datarep['Estado'] = datarep['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    datarep.loc[datarep['Estado'] != 'Activo','Estado']='Inactivo'
    datarep['Identidad'] = datarep['Identidad'].astype(str)
    datarep["Identidad"] = datarep["Identidad"].str.lower()
    datarep['Identidad'] = [ele.lstrip('.') for ele in datarep['Identidad']]
    datarep['Identidad'] = [ele.rstrip('.') for ele in datarep['Identidad']]
    datarep['Identidad'] = [ele.lstrip(' ') for ele in datarep['Identidad']]
    datarep['Identidad'] = [ele.rstrip(' ') for ele in datarep['Identidad']]
    filtroId = datarep[datarep.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    datarep = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    datarep.index = range(datarep.shape[0])
    datarep=datarep.assign(Sistema='MOTOR DE PAGOS') # se agrega una columna para saber que ese filtro fue de ClaroCo
    datarep=datarep.assign(Nombre_BD='DATAREP')
    datarep =datarep.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, datarep])
    Nube.index = range(Nube.shape[0])

POLIEDRO - ACTIVA.xlsx

In [ ]:
file_exists_activa = os.path.exists(filenameT+"\\POLIEDRO - ACTIVA.xlsx")

print(file_exists_activa)

if file_exists_activa == True:
    activa=pd.read_excel(filenameT+"\\POLIEDRO - ACTIVA.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    activa.drop(activa.index[0:11], inplace=True)
    activa= activa.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    activa['Estado'] = activa['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    activa['Estado'] = activa['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    activa['Estado'] = activa['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    activa.loc[activa['Estado'] != 'Activo','Estado']='Inactivo'
    activa['Identidad'] = activa['Identidad'].astype(str)
    activa["Identidad"] = activa["Identidad"].str.lower()
    activa['Identidad'] = [ele.lstrip('.') for ele in activa['Identidad']]
    activa['Identidad'] = [ele.rstrip('.') for ele in activa['Identidad']]
    activa['Identidad'] = [ele.lstrip(' ') for ele in activa['Identidad']]
    activa['Identidad'] = [ele.rstrip(' ') for ele in activa['Identidad']]
    filtroId = activa[activa.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    activa = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    activa.index = range(activa.shape[0])
    activa=activa.assign(Sistema='POLIEDRO') # se agrega una columna para saber que ese filtro fue de ClaroCo
    activa=activa.assign(Nombre_BD='ACTIVA')
    activa =activa.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, activa])
    Nube.index = range(Nube.shape[0])

POLIEDRO - POLIEXP

In [ ]:
file_exists_poliexp = os.path.exists(filenameT+"\\POLIEDRO - POLIEXP.xlsx")

print(file_exists_poliexp)

if file_exists_poliexp == True:
    poliexp=pd.read_excel(filenameT+"\\POLIEDRO - POLIEXP.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    poliexp.drop(poliexp.index[0:11], inplace=True)
    poliexp= poliexp.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    poliexp['Estado'] = poliexp['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    poliexp['Estado'] = poliexp['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    poliexp['Estado'] = poliexp['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    poliexp.loc[poliexp['Estado'] != 'Activo','Estado']='Inactivo'
    poliexp['Identidad'] = poliexp['Identidad'].astype(str)
    poliexp["Identidad"] = poliexp["Identidad"].str.lower()
    poliexp['Identidad'] = [ele.lstrip('.') for ele in poliexp['Identidad']]
    poliexp['Identidad'] = [ele.rstrip('.') for ele in poliexp['Identidad']]
    poliexp['Identidad'] = [ele.lstrip(' ') for ele in poliexp['Identidad']]
    poliexp['Identidad'] = [ele.rstrip(' ') for ele in poliexp['Identidad']]
    filtroId = poliexp[poliexp.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    poliexp = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    poliexp.index = range(poliexp.shape[0])
    poliexp=poliexp.assign(Sistema='POLIEDRO') # se agrega una columna para saber que ese filtro fue de ClaroCo
    poliexp=poliexp.assign(Nombre_BD='POLIEXP')
    poliexp =poliexp.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, poliexp])
    Nube.index = range(Nube.shape[0])

SGA - COFACPRD

In [ ]:
file_exists_cofacprd = os.path.exists(filenameT+"\\SGA - COFACPRD.xlsx")

print(file_exists_cofacprd)

if file_exists_cofacprd == True:
    cofacprd=pd.read_excel(filenameT+"\\SGA - COFACPRD.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    cofacprd.drop(cofacprd.index[0:11], inplace=True)
    cofacprd= cofacprd.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    cofacprd['Estado'] = cofacprd['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    cofacprd['Estado'] = cofacprd['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    cofacprd['Estado'] = cofacprd['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    cofacprd.loc[cofacprd['Estado'] != 'Activo','Estado']='Inactivo'
    cofacprd['Identidad'] = cofacprd['Identidad'].astype(str)
    cofacprd["Identidad"] = cofacprd["Identidad"].str.lower()
    cofacprd['Identidad'] = [ele.lstrip('.') for ele in cofacprd['Identidad']]
    cofacprd['Identidad'] = [ele.rstrip('.') for ele in cofacprd['Identidad']]
    cofacprd['Identidad'] = [ele.lstrip(' ') for ele in cofacprd['Identidad']]
    cofacprd['Identidad'] = [ele.rstrip(' ') for ele in cofacprd['Identidad']]
    filtroId = cofacprd[cofacprd.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    cofacprd = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    cofacprd.index = range(cofacprd.shape[0])
    cofacprd=cofacprd.assign(Sistema='SGA') # se agrega una columna para saber que ese filtro fue de ClaroCo
    cofacprd=cofacprd.assign(Nombre_BD='COFACPRD')
    cofacprd =cofacprd.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, cofacprd])
    Nube.index = range(Nube.shape[0])

SICACOM - CAJAS.xlsx

In [ ]:
file_exists_cajas = os.path.exists(filenameT+"\\SICACOM - CAJAS.xlsx")

print(file_exists_cajas)

if file_exists_cajas == True:
    cajas=pd.read_excel(filenameT+"\\SICACOM - CAJAS.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    cajas.drop(cajas.index[0:11], inplace=True)
    cajas= cajas.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    cajas['Estado'] = cajas['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    cajas['Estado'] = cajas['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    cajas['Estado'] = cajas['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    cajas.loc[cajas['Estado'] != 'Activo','Estado']='Inactivo'
    cajas['Identidad'] = cajas['Identidad'].astype(str)
    cajas["Identidad"] = cajas["Identidad"].str.lower()
    cajas['Identidad'] = [ele.lstrip('.') for ele in cajas['Identidad']]
    cajas['Identidad'] = [ele.rstrip('.') for ele in cajas['Identidad']]
    cajas['Identidad'] = [ele.lstrip(' ') for ele in cajas['Identidad']]
    cajas['Identidad'] = [ele.rstrip(' ') for ele in cajas['Identidad']]
    filtroId = cajas[cajas.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    cajas = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    cajas.index = range(cajas.shape[0])
    cajas=cajas.assign(Sistema='SICACOM') # se agrega una columna para saber que ese filtro fue de ClaroCo
    cajas=cajas.assign(Nombre_BD='CAJAS')
    cajas =cajas.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, cajas])
    Nube.index = range(Nube.shape[0])

NETEZZA PROD

In [ ]:
file_exists_Netezza = os.path.exists(filenameT+"\\NETEZZA.txt")

print(file_exists_Netezza)
if file_exists_Netezza == True:

    netezza=pd.read_csv(filenameT+"\\NETEZZA.txt", sep='|',encoding = "ISO-8859-1" , engine='python') #IMPORTAR
    netezza.columns = netezza.columns.str.strip()
    netezza['VALID_UNTIL'] = pd.to_datetime(netezza['VALID_UNTIL'], errors='coerce')

    fecha_actual = str(datetime.datetime.now().date())
    hora_actual = datetime.datetime.now().time()
    hora_actual = hora_actual.strftime('%H:%M:%S')
    actual=fecha_actual + ' ' +hora_actual
    netezza.loc[netezza['VALID_UNTIL'] < actual,'VALID_UNTIL']='Inactivo'
    netezza.loc[netezza['VALID_UNTIL'] != 'Inactivo','VALID_UNTIL']='Activo'
    netezza=netezza[['USERNAME','VALID_UNTIL']]
    netezza.rename(columns={'USERNAME':'Identidad', 'VALID_UNTIL':'Estado'}, inplace=True)
    netezza["Identidad"] = netezza["Identidad"].str.lower()
    netezza['Identidad'] = [ele.lstrip('.') for ele in netezza['Identidad']]
    netezza['Identidad'] = [ele.rstrip('.') for ele in netezza['Identidad']]
    netezza['Identidad'] = [ele.lstrip(' ') for ele in netezza['Identidad']]
    netezza['Identidad'] = [ele.rstrip(' ') for ele in netezza['Identidad']]
    filtroId = netezza[netezza.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    netezza = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    netezza.index = range(netezza.shape[0])
    netezza=netezza.assign(Sistema='NETEZZA') # se agrega una columna para saber que ese filtro fue de ClaroCo
    netezza=netezza.assign(Nombre_BD='NETEZZA')
    netezza =netezza.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, netezza])
    Nube.index = range(Nube.shape[0])

EPA

In [ ]:
file_exists_Epa = os.path.exists(filenameT+"\\EPA.csv")

print(file_exists_Epa)
if file_exists_Epa == True:

    epa=pd.read_csv(filenameT+"\\EPA.csv", sep=';',encoding = "utf-8-sig" , engine='python') #IMPORTAR
    epa.columns = epa.columns.str.strip()

    epa=epa[['USERNAME','ACCOUNT_STATUS']]
    epa.rename(columns={'USERNAME':'Identidad', 'ACCOUNT_STATUS':'Estado'}, inplace=True)
    epa['Identidad'] = epa['Identidad'].astype(str)
    epa["Identidad"] = epa["Identidad"].str.lower()
    epa['Identidad'] = [ele.lstrip('.') for ele in epa['Identidad']]
    epa['Identidad'] = [ele.rstrip('.') for ele in epa['Identidad']]
    epa['Identidad'] = [ele.lstrip(' ') for ele in epa['Identidad']]
    epa['Identidad'] = [ele.rstrip(' ') for ele in epa['Identidad']]
    filtroId = epa[epa.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    epa = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    epa.index = range(epa.shape[0])
    epa=epa.assign(Sistema='SAP 6.0') # se agrega una columna para saber que ese filtro fue de ClaroCo
    epa=epa.assign(Nombre_BD='EPA')
    epa =epa.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, epa])
    Nube.index = range(Nube.shape[0])

MAXIMO - CTGINS1_exacc101vm01

In [ ]:
file_exists_ctgin = os.path.exists(filenameT+"\\MAXIMO - CTGINS1_exacc101vm01.xlsx")

print(file_exists_ctgin)

if file_exists_ctgin == True:
    ctgin=pd.read_excel(filenameT+"\\MAXIMO - CTGINS1_exacc101vm01.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    ctgin.drop(ctgin.index[0:11], inplace=True)
    ctgin= ctgin.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    ctgin['Estado'] = ctgin['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    ctgin['Estado'] = ctgin['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    ctgin['Estado'] = ctgin['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    ctgin.loc[ctgin['Estado'] != 'Activo','Estado']='Inactivo'
    ctgin['Identidad'] = ctgin['Identidad'].astype(str)
    ctgin["Identidad"] = ctgin["Identidad"].str.lower()
    ctgin['Identidad'] = [ele.lstrip('.') for ele in ctgin['Identidad']]
    ctgin['Identidad'] = [ele.rstrip('.') for ele in ctgin['Identidad']]
    ctgin['Identidad'] = [ele.lstrip(' ') for ele in ctgin['Identidad']]
    ctgin['Identidad'] = [ele.rstrip(' ') for ele in ctgin['Identidad']]
    filtroId = ctgin[ctgin.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    ctgin = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    ctgin.index = range(ctgin.shape[0])
    ctgin=ctgin.assign(Sistema='MAXIMO') # se agrega una columna para saber que ese filtro fue de ClaroCo
    ctgin=ctgin.assign(Nombre_BD='CTGINS1_exacc101vm01')
    ctgin =ctgin.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, ctgin])
    Nube.index = range(Nube.shape[0])

REMPDB

In [ ]:
file_exists_rempdb = os.path.exists(filenameT+"\\MYIT - REMPDB.xlsx")

print(file_exists_rempdb)

if file_exists_rempdb == True:
    rempdb=pd.read_excel(filenameT+"\\MYIT - REMPDB.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    rempdb.drop(rempdb.index[0:11], inplace=True)
    rempdb= rempdb.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    rempdb['Estado'] = rempdb['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    rempdb['Estado'] = rempdb['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    rempdb['Estado'] = rempdb['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    rempdb.loc[rempdb['Estado'] != 'Activo','Estado']='Inactivo'
    rempdb['Identidad'] = rempdb['Identidad'].astype(str)
    rempdb["Identidad"] = rempdb["Identidad"].str.lower()
    rempdb['Identidad'] = [ele.lstrip('.') for ele in rempdb['Identidad']]
    rempdb['Identidad'] = [ele.rstrip('.') for ele in rempdb['Identidad']]
    rempdb['Identidad'] = [ele.lstrip(' ') for ele in rempdb['Identidad']]
    rempdb['Identidad'] = [ele.rstrip(' ') for ele in rempdb['Identidad']]
    filtroId = rempdb[rempdb.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    rempdb = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    rempdb.index = range(rempdb.shape[0])
    rempdb=rempdb.assign(Sistema='MYIT') # se agrega una columna para saber que ese filtro fue de ClaroCo
    rempdb=rempdb.assign(Nombre_BD='REMPDB')
    rempdb =rempdb.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, rempdb])
    Nube.index = range(Nube.shape[0])

PDB_SMARTIT

In [ ]:
file_exists_smartit = os.path.exists(filenameT+"\\MYIT - PDB_SMARTIT.xlsx")

print(file_exists_smartit)

if file_exists_smartit == True:
    smartit=pd.read_excel(filenameT+"\\MYIT - PDB_SMARTIT.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    smartit.drop(smartit.index[0:11], inplace=True)
    smartit= smartit.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    smartit['Estado'] = smartit['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    smartit['Estado'] = smartit['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    smartit['Estado'] = smartit['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    smartit.loc[smartit['Estado'] != 'Activo','Estado']='Inactivo'
    smartit['Identidad'] = smartit['Identidad'].astype(str)
    smartit["Identidad"] = smartit["Identidad"].str.lower()
    smartit['Identidad'] = [ele.lstrip('.') for ele in smartit['Identidad']]
    smartit['Identidad'] = [ele.rstrip('.') for ele in smartit['Identidad']]
    smartit['Identidad'] = [ele.lstrip(' ') for ele in smartit['Identidad']]
    smartit['Identidad'] = [ele.rstrip(' ') for ele in smartit['Identidad']]
    filtroId = smartit[smartit.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    smartit = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    smartit.index = range(smartit.shape[0])
    smartit=smartit.assign(Sistema='MYIT') # se agrega una columna para saber que ese filtro fue de ClaroCo
    smartit=smartit.assign(Nombre_BD='SMARTIT')
    smartit =smartit.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, smartit])
    Nube.index = range(Nube.shape[0])

MYIT - REPPDB.xlsx

In [ ]:
file_exists_reppdb = os.path.exists(filenameT+"\\MYIT - REPPDB.xlsx")

print(file_exists_reppdb)

if file_exists_reppdb == True:
    reppdb=pd.read_excel(filenameT+"\\MYIT - REPPDB.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    reppdb.drop(reppdb.index[0:11], inplace=True)
    reppdb= reppdb.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    reppdb['Estado'] = reppdb['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    reppdb['Estado'] = reppdb['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    reppdb['Estado'] = reppdb['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    reppdb.loc[reppdb['Estado'] != 'Activo','Estado']='Inactivo'
    reppdb['Identidad'] = reppdb['Identidad'].astype(str)
    reppdb["Identidad"] = reppdb["Identidad"].str.lower()
    reppdb['Identidad'] = [ele.lstrip('.') for ele in reppdb['Identidad']]
    reppdb['Identidad'] = [ele.rstrip('.') for ele in reppdb['Identidad']]
    reppdb['Identidad'] = [ele.lstrip(' ') for ele in reppdb['Identidad']]
    reppdb['Identidad'] = [ele.rstrip(' ') for ele in reppdb['Identidad']]
    filtroId = reppdb[reppdb.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    reppdb = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    reppdb.index = range(reppdb.shape[0])
    reppdb=reppdb.assign(Sistema='MYIT') # se agrega una columna para saber que ese filtro fue de ClaroCo
    reppdb=reppdb.assign(Nombre_BD='REPPDB')
    reppdb =reppdb.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, reppdb])
    Nube.index = range(Nube.shape[0])

MYIT - DWPPDB.xlsx

In [ ]:
file_exists_dwppdb = os.path.exists(filenameT+"\\MYIT - DWPPDB.xlsx")

print(file_exists_dwppdb)

if file_exists_dwppdb == True:
    dwppdb=pd.read_excel(filenameT+"\\MYIT - DWPPDB.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    dwppdb.drop(dwppdb.index[0:11], inplace=True)
    dwppdb= dwppdb.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    dwppdb['Estado'] = dwppdb['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    dwppdb['Estado'] = dwppdb['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    dwppdb['Estado'] = dwppdb['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    dwppdb.loc[dwppdb['Estado'] != 'Activo','Estado']='Inactivo'
    dwppdb['Identidad'] = dwppdb['Identidad'].astype(str)
    dwppdb["Identidad"] = dwppdb["Identidad"].str.lower()
    dwppdb['Identidad'] = [ele.lstrip('.') for ele in dwppdb['Identidad']]
    dwppdb['Identidad'] = [ele.rstrip('.') for ele in dwppdb['Identidad']]
    dwppdb['Identidad'] = [ele.lstrip(' ') for ele in dwppdb['Identidad']]
    dwppdb['Identidad'] = [ele.rstrip(' ') for ele in dwppdb['Identidad']]
    filtroId = dwppdb[dwppdb.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    dwppdb = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    dwppdb.index = range(dwppdb.shape[0])
    dwppdb=dwppdb.assign(Sistema='MYIT') # se agrega una columna para saber que ese filtro fue de ClaroCo
    dwppdb=dwppdb.assign(Nombre_BD='DWPPDB')
    dwppdb =dwppdb.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, dwppdb])
    Nube.index = range(Nube.shape[0])

MYIT - PDB_DWP.xlsx

In [ ]:
file_exists_pdb_dwp = os.path.exists(filenameT+"\\MYIT - PDB_DWP.xlsx")

print(file_exists_pdb_dwp)

if file_exists_pdb_dwp == True:
    pdbdwp=pd.read_excel(filenameT+"\\MYIT - PDB_DWP.xlsx",names=['Identidad', '2', 'Estado', '4','5','6','7','8','9','10'])
    pdbdwp.drop(pdbdwp.index[0:11], inplace=True)
    pdbdwp= pdbdwp.drop(['2', '4', '5', '6', '7', '8', '9', '9', '10'], axis=1)
    pdbdwp['Estado'] = pdbdwp['Estado'].astype(str).str.replace("OPEN", "Activo", regex=True)
    pdbdwp['Estado'] = pdbdwp['Estado'].astype(str).str.replace("EXPIRED(GRACE)", "Activo", regex=True)
    pdbdwp['Estado'] = pdbdwp['Estado'].astype(str).str.replace("EXPIRED", "Activo", regex=True)
    pdbdwp.loc[pdbdwp['Estado'] != 'Activo','Estado']='Inactivo'
    pdbdwp['Identidad'] = pdbdwp['Identidad'].astype(str)
    pdbdwp["Identidad"] = pdbdwp["Identidad"].str.lower()
    pdbdwp['Identidad'] = [ele.lstrip('.') for ele in pdbdwp['Identidad']]
    pdbdwp['Identidad'] = [ele.rstrip('.') for ele in pdbdwp['Identidad']]
    pdbdwp['Identidad'] = [ele.lstrip(' ') for ele in pdbdwp['Identidad']]
    pdbdwp['Identidad'] = [ele.rstrip(' ') for ele in pdbdwp['Identidad']]
    filtroId = pdbdwp[pdbdwp.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    pdbdwp = filtroId.merge(Usuarios, how='left', left_on='Identidad', right_on='Identidad')
    pdbdwp.index = range(pdbdwp.shape[0])
    pdbdwp=pdbdwp.assign(Sistema='MYIT') # se agrega una columna para saber que ese filtro fue de ClaroCo
    pdbdwp=pdbdwp.assign(Nombre_BD='PDB_DWP')
    pdbdwp =pdbdwp.assign(Nombre='Sin nombre')
    Nube = pd.concat([Nube, pdbdwp])
    Nube.index = range(Nube.shape[0])

In [ ]:
Nube

In [ ]:
file_exists_SSFF_Activos = os.path.exists(filenameS+"\\Informe_Activos_Claro.xlsx")#Pregunta si en la ruta existe el archivo

print(file_exists_SSFF_Activos)#muestra false o true 

if file_exists_SSFF_Activos == True:#Si el archivo existe

    SsffA=pd.read_excel(filenameS+"\\Informe_Activos_Claro.xlsx")
    SsffA.columns = SsffA.columns.str.strip()
    SsffA=SsffA[['Cédula','Nombres', 'Apellidos' ,'Detalles de empleo Fecha de ingreso','Estado de colaborador']] #Usar solo las columnas que necesitamos
    SsffA.rename(columns={'Estado de colaborador':'STATUS SSFF', 'Cédula':'CUSTOM08', 'Detalles de empleo Fecha de ingreso':'Fecha Ingreso SSFF'}, inplace=True)
    SsffA.insert(loc = 5,column = 'Fecha Terminacion SSFF',value = '0')
    SsffA=SsffA[['STATUS SSFF',  'CUSTOM08', 'Fecha Ingreso SSFF', 'Fecha Terminacion SSFF','Nombres', 'Apellidos']]

    SsffA = SsffA.sort_values('Fecha Ingreso SSFF', ascending=False)
    SsffA['Fecha Ingreso SSFF'] = pd.to_datetime(SsffA['Fecha Ingreso SSFF'], format="%d/%m/%Y %H:%M:%S" , errors='coerce')
    SsffA['Fecha Ingreso SSFF'] = SsffA['Fecha Ingreso SSFF'].dt.floor('D')
    pd.options.display.float_format = '{:.0f}'.format
    SsffA['CUSTOM08']= SsffA['CUSTOM08'].astype('Int64')

    SsffA['CUSTOM08'] = SsffA['CUSTOM08'].astype(str)
    SsffA['Nombres'] = SsffA['Nombres'].astype(str)
    SsffA['Apellidos'] = SsffA['Apellidos'].astype(str)
    
    SsffA = SsffA.dropna(subset=["CUSTOM08"])
    
    SsffA['CUSTOM08'] = [ele.lstrip('0') for ele in SsffA['CUSTOM08']]
    SsffA['CUSTOM08'] = [ele.lstrip(' ') for ele in SsffA['CUSTOM08']]
    SsffA['CUSTOM08'] = [ele.rstrip(' ') for ele in SsffA['CUSTOM08']]

    SsffA['Nombres'] = [ele.lstrip(' ') for ele in SsffA['Nombres']]
    SsffA['Apellidos'] = [ele.lstrip(' ') for ele in SsffA['Apellidos']]

    
    SsffA = SsffA.dropna(subset=["Nombres"])

    SsffA=SsffA.drop_duplicates(SsffA.columns[~SsffA.columns.isin(['Fecha Ingreso SSFF'])], keep='first')
    Ssff1=SsffA[['STATUS SSFF','CUSTOM08','Nombres', 'Apellidos']]
    Ssff1.rename(columns={'STATUS SSFF':'Estado'}, inplace=True)
    SsffA=SsffA[['STATUS SSFF',  'CUSTOM08', 'Fecha Ingreso SSFF', 'Fecha Terminacion SSFF']]



    Nube = Nube.merge(SsffA, how='left', left_on='DocumentoRRHH', right_on='CUSTOM08')
    Nube = Nube.fillna({'STATUS SSFF': 'Inactivo', 'Fecha Terminacion SSFF': '0'}) 
    
    Nube['Fecha Terminacion SSFF'] = pd.to_datetime(Nube['Fecha Terminacion SSFF'], format="%d/%m/%Y" , errors='ignore')
    Nube['FECHA DE RETIRO'] = pd.to_datetime(Nube['FECHA DE RETIRO'], format="%Y/%m/%d" , errors='coerce')


    Nube = Nube.drop('CUSTOM08', axis=1)
    Nube=Nube.drop_duplicates(Nube.columns[~Nube.columns.isin(['STATUS SSFF', 'Fecha Ingreso SSFF', 'Fecha Terminacion SSFF'])], keep='first')


In [ ]:
Nube

In [ ]:
file_exists_CA = os.path.exists(filenameA+"\\Export CA.xlsx")

print(file_exists_CA)
if file_exists_CA == True:

    CA=pd.read_excel(filenameA+"\\Export CA.xlsx", sheet_name=0) #IMPORTAR
    CA=CA[['Identidad']] #USAR SOLO ESAS TRES COLUMNAS
    CA['Identidad'] = CA['Identidad'].astype(str)
    CA['Identidad'] = [ele.lstrip(' ') for ele in CA['Identidad']]
    CA['Identidad'] = [ele.rstrip(' ') for ele in CA['Identidad']]
    CA['Identidad'] = CA['Identidad'].str.lower()
    identidadesCA=CA['Identidad'].to_list()


In [ ]:
fecha_actual = datetime.datetime.now().date()
hora_actual = datetime.datetime.now().time()
hora_actual = hora_actual.strftime('%H/%M/%S')
ayer = str(fecha_actual - datetime.timedelta(days=1))

In [ ]:
aplicativos = {'LNXRACDBPR01':'CA', 'OIM1':'OIM','BSCSPROD':'AC - BSCS - MOTOR DE PAGOS', 'CALDMPROD':'CALLIDUS', 
    'TRUECPROD':'CALLIDUS', 'LOMBAPROD':'CALLIDUS',
    'BUSOBPROD':'CALLIDUS', 'ONYX':'CRM-ONYX', 'WBMSDBPR' : 'INTEC', 'INTEGPRD 70':'INTEGRADOR WEB', 'INTEGPRD 71':'INTEGRADOR WEB',
    'GPRS': 'MAGUARE', 'PRTPCOL1_COMIBM108':'MAHINDRA', 'PRTPCOL2_COMIBM107':'MAHINDRA', 'USSDCOL1':'MAHINDRA','USSDCOL2':'MAHINDRA', 
    'DATAREP':'MOTOR DE PAGOS', 'ACTIVA':'POLIEDRO', 'POLIEXP':'POLIEDRO', 'COFACPRD':'SGA', 'CAJAS':'SICACOM', 
    'NETEZZA':'NETEZZA', 'EPA':'SAP 6.0', 'CTGINS1_exacc101vm01':'MAXIMO', 'REMPDB':'MYIT',
    'SMARTIT':'MYIT', 'REPPDB':'MYIT', 'DWPPDB':'MYIT' , 'PDB_DWP':'MYIT',  'AUTOMIC':'AZURE DEVOPS'}
for c in aplicativos: 
    sumame=Nube.apply(lambda x: x['Nombre_BD'] == c , axis=1).sum()
    if sumame == 0:
        datos = {'Identidad':['Sin identidad'],'Nombre':['Sin Nombre'],'Estado':['N/E'],
        'Cruce':['No cruza'],'DocumentoRRHH':['0'],'Sistema': aplicativos[c],'Nombre_BD':[c],
        'NombreRRHH':['Sin Nombre'],'FECHA DE RETIRO':['0'], 'CASO':['Sin caso'],
        'TIPO DE CONTRATO':['Sin contrato'] , 'STATUS SSFF':['N/E']}
        aplicativo = pd.DataFrame(datos)
        Nube = pd.concat([Nube, aplicativo])
Nube = Nube.fillna({'NombreRRHH': 'Sin Nombre', 'FECHA DE RETIRO': 'N/A', 'Fecha Ingreso SSFF':'0' ,'Fecha Terminacion SSFF':'0','CASO': 'Sin caso', 'STATUS SSFF':'N/E',
'coincidencia nombre APP vs SSFF':'0%'})
Nube.index = range(Nube.shape[0])

In [ ]:
Nube = Nube.fillna({'Nombre': 'Sin Nombre','Identidad': 'Sin identidad'})
Nube.loc[(Nube['Nombre'].isnull()) | (Nube['Nombre']=="") | (Nube['Nombre'].isna()) | (Nube['Nombre']=="nan"), 'Nombre']='Sin Nombre'
Nube.loc[(Nube['Identidad'].isnull()) | (Nube['Identidad']=="") | (Nube['Identidad'].isna()), 'Identidad']='Sin identidad'

In [ ]:
Nube['Nombre'] = Nube['Nombre'].astype(str).str.replace("ñ", "n", regex=True)
Nube['NombreRRHH'] = Nube['NombreRRHH'].astype(str).str.replace("ñ", "n", regex=True)
name = Nube["NombreRRHH"].str.split( n=4, expand=True)
name.columns = ['nombre1', 'nombre2','apellido1', 'apellido2', 'quintobueno']
name = name.fillna({'nombre1': '1', 'nombre2': '2','apellido1': '3', 'apellido2': '4', 'quintobueno': '5'})
Nube=pd.concat([ Nube, name], axis=1)
def match_string(NombreGrande, nombreChiquito):
    s = NombreGrande.find(nombreChiquito)
    if s == -1:
        return "No encontrado"
    else:
        return s

Nube['Nombre1_exist'] = Nube.apply(lambda x: match_string(x['Nombre'],x['nombre1']), axis=1)
Nube['Nombre2_exist'] = Nube.apply(lambda x: match_string(x['Nombre'],x['nombre2']), axis=1)
Nube['Apellido1_exist'] = Nube.apply(lambda x: match_string(x['Nombre'],x['apellido1']), axis=1)
Nube['Apellido2_exist'] = Nube.apply(lambda x: match_string(x['Nombre'],x['apellido2']), axis=1)
Nube['Quinto_exist'] = Nube.apply(lambda x: match_string(x['Nombre'],x['quintobueno']), axis=1)

In [ ]:
def sumarCruces(nom1,nom2,ape1,ape2, quinto):
    suma=0
    if nom1 != "No encontrado":
        suma=suma+25
    if nom2 != "No encontrado":
        suma=suma+25
    if ape1 != "No encontrado":
        suma=suma+25
    if ape2 != "No encontrado":
        suma=suma+25
    if quinto != "No encontrado":
        suma=suma+25
    return suma

Nube['coincidencia nombre APP vs SSFF'] = Nube.apply(lambda x: sumarCruces(x['Nombre1_exist'],x['Nombre2_exist'],x['Apellido1_exist'],x['Apellido2_exist'],x['Quinto_exist']), axis=1)
Nube = Nube.drop(['nombre1','nombre2','apellido1',  'apellido2','Nombre1_exist','Nombre2_exist','Apellido1_exist','Apellido2_exist','quintobueno','Quinto_exist' ], axis=1)

In [ ]:
Nube=Nube.assign(Espacios=Nube['Nombre'].str.count(' '))
Nube.insert(loc=14, column='Aplica Inactivacion', value  = 'si')
Nube.loc[(Nube['Cruce'] == 'Identidad')& (Nube['Nombre'] != 'Sin Nombre') & (Nube['coincidencia nombre APP vs SSFF'] <50) , 'Aplica Inactivacion']='no'
Nube.loc[(Nube['Cruce'] == 'Identidad')& (Nube['Nombre'] != 'Sin Nombre') & (Nube['coincidencia nombre APP vs SSFF'] ==0) , 'Aplica Inactivacion']='no'
Nube.loc[(Nube['Cruce'] == 'Identidad')& (Nube['Nombre'] != 'Sin Nombre')& (Nube['coincidencia nombre APP vs SSFF'] ==25)&  (Nube['Espacios']==0), 'Aplica Inactivacion']='si'
Nube['coincidencia nombre APP vs SSFF']=Nube['coincidencia nombre APP vs SSFF'].astype(str)+"%"
Nube = Nube.drop('Espacios', axis=1)
def comprobarNombre (nombre):
    nombrebb = ['csm', 'consultor','call ', 'asesor', 'cav ', 'eliminacion',  'gestor', 'ventas', 'analista', 'libre', 'people',
                 'pospago', 'telem', 'representante',
                            'None', 'analista', 'supervisor', 'colombia', 'contact', 'coordinador', 'servicio', 'personalizado', 'clientes']
    for c in nombrebb:
        print(c)
        s = nombre.find(c)
        if s != -1:
            return "si"
Nube['contiene'] = Nube.apply(lambda x: comprobarNombre(x['Nombre']), axis=1)
Nube.loc[(Nube['Cruce'] == 'Identidad')& (Nube['Nombre'] != 'Sin Nombre')& (Nube['Nombre'].notnull())& (Nube['contiene']=="si"), 'Aplica Inactivacion']='si'
Nube.loc[(Nube['Nombre_BD'] != 'BSCSPROD'), 'Aplica Inactivacion']='si'
Nube = Nube.drop('contiene', axis=1)

In [ ]:
Nube=Nube.assign(Reintegro='N/A')
if file_exists_CA == True:
    Nube['Existe en CA'] = Nube['Identidad'].apply(lambda x: "si" if x in identidadesCA else "no")

In [ ]:
fecha_actual = str(datetime.datetime.now().date())
hora_actual = datetime.datetime.now().time()
hora_actual = hora_actual.strftime('%H-%M-%S')

nombref=filename+'\\IdentidadesBDconSSFF '+fecha_actual+' '+hora_actual+'.csv'
Nube.to_csv(nombref,index=False,sep=';',encoding='utf-8-sig')
nombre1=filename+'\\IdentidadesBDconSSFF.csv'
Nube.to_csv(nombre1,index=False,sep=';',encoding='utf-8-sig')

In [ ]:
Nube